In [3]:
import azureml.core
from azureml.core import Workspace
from azureml.pipeline.core import Pipeline, PublishedPipeline
from azureml.core.experiment import Experiment

ws = Workspace.from_config()

experiments = Experiment.list(ws)
for experiment in experiments:
    print(experiment.name)

published_pipelines = PublishedPipeline.list(ws)
for published_pipeline in  published_pipelines:
    print(f"{published_pipeline.name},'{published_pipeline.id}'")

experiment_name = "MyExperiment" 
pipeline_id = "aaaaaaaaa-bbbb-cccc-dddd-eeeeeeeeeeee"

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


prepare_image
test-experiment
ws-azureml-atnan
randomforest-iris-1,'e300597d-23d0-4cac-8426-b73b62c576d8'
randomforest-iris-2,'6db00887-f286-44d0-bad6-0dae4cc58f32'
randomforest-iris-1,'e51d224a-8421-4652-b7e6-f00114d72fe2'
randomforest-iris-2,'520b10ee-db1e-4182-aec7-466643df416c'
randomforest-iris-1,'f50bc30a-981f-4ff0-bd7b-a1383bcc62ff'


In [6]:
published_pipelines

[Pipeline(Name: randomforest-iris-1,
 Id: e300597d-23d0-4cac-8426-b73b62c576d8,
 Status: Active,
 Endpoint: https://westus.api.azureml.ms/pipelines/v1.0/subscriptions/f4550262-c232-4d1f-8f2f-70bcd32ebfe1/resourceGroups/rg-azureml/providers/Microsoft.MachineLearningServices/workspaces/ws-azureml/PipelineRuns/PipelineSubmit/e300597d-23d0-4cac-8426-b73b62c576d8),
 Pipeline(Name: randomforest-iris-2,
 Id: 6db00887-f286-44d0-bad6-0dae4cc58f32,
 Status: Active,
 Endpoint: https://westus.api.azureml.ms/pipelines/v1.0/subscriptions/f4550262-c232-4d1f-8f2f-70bcd32ebfe1/resourceGroups/rg-azureml/providers/Microsoft.MachineLearningServices/workspaces/ws-azureml/PipelineRuns/PipelineSubmit/6db00887-f286-44d0-bad6-0dae4cc58f32),
 Pipeline(Name: randomforest-iris-1,
 Id: e51d224a-8421-4652-b7e6-f00114d72fe2,
 Status: Active,
 Endpoint: https://westus.api.azureml.ms/pipelines/v1.0/subscriptions/f4550262-c232-4d1f-8f2f-70bcd32ebfe1/resourceGroups/rg-azureml/providers/Microsoft.MachineLearningServices/

In [7]:
# You can find the pipeline ID in Azure Machine Learning studio

published_pipeline = PublishedPipeline.get(ws, id="e300597d-23d0-4cac-8426-b73b62c576d8")
published_pipeline.endpoint

'https://westus.api.azureml.ms/pipelines/v1.0/subscriptions/f4550262-c232-4d1f-8f2f-70bcd32ebfe1/resourceGroups/rg-azureml/providers/Microsoft.MachineLearningServices/workspaces/ws-azureml/PipelineRuns/PipelineSubmit/e300597d-23d0-4cac-8426-b73b62c576d8'

In [ ]:
from azure.ai.ml import MLClient, command, Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

data_asset = ml_client.data.get("iris", version="1")

# to successfully create a job, customize the parameters below based on your workspace resources
job = command(
        command='ls "${{inputs.data}}"',
        inputs={
            "data": Input(path=data_asset.id,
                type=AssetTypes.URI_FOLDER,
                mode=InputOutputModes.RO_MOUNT
            )
        },
        environment="azureml:AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest"
      )
returned_job = ml_client.jobs.create_or_update(job)